In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader("data/chase_banking.pdf")
documents= loader.load()
print(documents)

[Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.5 (Macintosh)', 'creationdate': '2024-10-03T09:29:06-04:00', 'author': 'JPMorgan Chase Bank', 'keywords': 'Chase; total; checking; guide to your account; ada; (PDF)', 'moddate': '2024-10-07T09:59:35-04:00', 'subject': 'Chase Total Checking - A Guide To Your Account', 'title': 'Chase Total Checking - A Guide To Your Account (PDF)', 'trapped': '/Unknown', 'source': 'data/chase_banking.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='HAVE QUESTIONS? CALL US AT 1-800-935-9935 (WE ACCEPT OPERATOR RELAY CALLS)  • WANT MORE INFO?  SEE THE DEPOSIT ACCOUNT AGREEMENT\n1\nCHASE TOTAL CHECKING\n®\nA GUIDE TO YOUR ACCOUNT †\nIt’s important that you understand how your Chase Total Checking account works. \nWe’ve created this Guide to explain the fees and some key terms of your personal account.\nMONTHLY \nSERVICE FEE*\nMonthly Service Fee $12\nWays to Avoid the \nMonthly Service Fee\n$0 Monthl

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
all_chunks = text_splitter.split_documents(documents)

In [5]:
import numpy as np
from langchain_community.embeddings import OllamaEmbeddings

# Initialize the Ollama embedding model
embedding_model = OllamaEmbeddings(model="mxbai-embed-large")

# Get the embeddings for all chunks
chunk_texts = [chunk.page_content for chunk in all_chunks]
chunk_embeddings_list = embedding_model.embed_documents(chunk_texts)

In [7]:
print(chunk_embeddings_list)

[[-0.009352479130029678, 0.11099083721637726, 0.12458088248968124, 0.5793946981430054, 0.037772297859191895, 0.10146096348762512, 0.24292001128196716, 0.026016950607299805, 0.4704999625682831, 0.8492102026939392, 0.6471679210662842, 0.5008590221405029, -0.03634458780288696, 0.06403874605894089, -1.027329921722412, 0.01782407984137535, -0.2137085199356079, -0.037720825523138046, -0.09689395129680634, -0.18226835131645203, -0.6652310490608215, -0.22710193693637848, -0.5289626121520996, -0.2075306475162506, -0.9397529363632202, 0.3106900453567505, -0.05974704399704933, -0.03077007830142975, 1.1048187017440796, 0.5948246717453003, -0.4853595793247223, 0.22521355748176575, 0.15240469574928284, 0.004148600623011589, -0.04992280527949333, -0.0606694258749485, 0.7466481328010559, -0.6923759579658508, -0.5685860514640808, -0.29008281230926514, -0.32568246126174927, 0.48916077613830566, 1.4246569871902466, -0.18016965687274933, -0.9203420281410217, -0.4792279899120331, -0.6733104586601257, -0.66

In [6]:
import faiss

# Convert the list of embeddings to a NumPy array, which Faiss requires
embeddings_array = np.array(chunk_embeddings_list).astype('float32')

# Get the dimension of the embedding vectors
dimension = embeddings_array.shape[1]

# --- IVF-specific setup ---
# Number of clusters for the IVF index. A good starting point is usually sqrt(N)
# N is the number of vectors, so we'll use a rule of thumb.
num_clusters = int(np.sqrt(len(embeddings_array)))
print(f"Creating an IVF index with {num_clusters} clusters.")

# The quantizer is the index for the cluster centroids
quantizer = faiss.IndexFlatL2(dimension)

# The IVF index itself. It takes the quantizer, dimension, and number of clusters.
index = faiss.IndexIVFFlat(quantizer, dimension, num_clusters, faiss.METRIC_L2)

# Train the index. This is where the algorithm finds the clusters.
print("Training the Faiss IVF index...")
index.train(embeddings_array)

# Add the embeddings to the trained index
print("Adding embeddings to the index...")
index.add(embeddings_array)

print("Faiss IVF index created and populated successfully.")

# To perform a search, you would convert your query to an embedding and then
# search the index.
# query_embedding = embedding_model.embed_query("what is the company policy?")
# distances, indices = index.search(np.array([query_embedding]).astype('float32'), k=5)

# The 'indices' variable would give you the index numbers of the most relevant chunks,
# which you can use to retrieve the original text from your 'all_chunks' list.

Creating an IVF index with 4 clusters.
Training the Faiss IVF index...
Adding embeddings to the index...
Faiss IVF index created and populated successfully.


In [8]:
# Your code to create the query embedding
question_vector = openai_client.embeddings.create(
    input='how to avoid overdraft',
    model=model
)
question_embeddings = question_vector.data[0].embedding

# The IVF search step
# Assume 'index' is your trained Faiss IVF index from our previous example
# and you have your embeddings stored as a NumPy array.

import numpy as np

# Convert the query embedding to a NumPy array as Faiss requires
query_vector_np = np.array([question_embeddings]).astype('float32')

# Perform the search on the index
# k=5 means we want the 5 most similar results
distances, indices = index.search(query_vector_np, k=5)

print("Found the 5 most relevant document indices:", indices)
print("Distances to these documents:", distances)

# You would then use these indices to retrieve the original text chunks
# from your list of documents to feed into an LLM.

NameError: name 'openai_client' is not defined